In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Masking, Dropout, InputLayer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
dataset = pd.read_csv("../data/archive/TranslateTest.csv", sep=";", encoding="utf-8")

In [3]:
dataset = dataset.fillna(" ")

In [4]:
dataset['Симптомы'] = dataset[["Симптом 1","Симптом 2","Симптом 3","Симптом 4","Симптом 5","Симптом 6","Симптом 7","Симптом 8","Симптом 9",
                               "Симптом 10","Симптом 11","Симптом 12","Симптом 13","Симптом 14","Симптом 15","Симптом 16","Симптом 17"]].agg(' '.join, axis=1)

# Удаляем лишние колонки
dataset = dataset[['Болезнь', 'Симптомы']]

dataset.to_csv("../data/archive/example.csv")

# Токинезация симптомов, то есть представление их в виде множества чисел
max_words_count=500
tokenizer=Tokenizer(num_words=max_words_count, split=" ")
tokenizer.fit_on_texts(dataset['Симптомы'])

# "Подгоняем" размер множеств чисел под одну длину 
data_x = tokenizer.texts_to_sequences(dataset['Симптомы'])

max_len = max([len(x) for x in data_x])
print(max_len)
data_x = pad_sequences(data_x, maxlen=max_len)

# Кодирование меток заболеваний
label_encoder = LabelEncoder()
data_y = label_encoder.fit_transform(dataset['Болезнь'])

30


In [7]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score



classifiers = [
    KNeighborsClassifier(5),
    SVC(probability=True, kernel='rbf'),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GaussianNB(),
    LogisticRegression(),
]

log_cols = ["Classifier", "Accuracy"]
log = pd.DataFrame(columns=log_cols)

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.2, random_state = 0)
acc_dict = {}

for clf in classifiers:
	name = clf.__class__.__name__
	clf.fit(X_train, y_train)
	train_predictions = clf.predict(X_test)
	acc = accuracy_score(y_test, train_predictions)
	if name in acc_dict:
		acc_dict[name] += acc
	else:
		acc_dict[name] = acc

for clf in acc_dict:
	acc_dict[clf] = acc_dict[clf] / 10.0
	log_entry = pd.DataFrame([[clf, acc_dict[clf]]], columns=log_cols)
	log = log.append(log_entry)

plt.xlabel('Accuracy')
plt.title('Classifier Accuracy')

sns.set_color_codes("muted")
sns.barplot(x='Accuracy', y='Classifier', data=log.sort_values(by='Accuracy'), color="b")

c:\Универ\4 курс\Диплом\code\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AttributeError: 'DataFrame' object has no attribute 'append'